# OIC-158 TEM Nuclei Segmentation
Packages to import

In [47]:
import napari
import numpy as np
import pyclesperanto as cle
import skimage as sk
from skimage.io import imread, imsave
import pandas as pd
import os
from glob import glob
import matplotlib as mpl
import matplotlib.pyplot as plt

All functions for pipeline

In [48]:
#Create a 25-pixel wide band along perimeter of nucleus
def band_25_pixels(masks,footprint):
    eroded = [sk.morphology.erosion(mask, footprint=footprint) for mask in masks]
    bands = [masks[i] - eroded[i] for i in range(len(masks))]
    return bands

#Threshold the dense regions within whole image
def norm_and_threshold(imgs,max_hole=100,min_object=20,match_img=3):
    adpt_norm = [sk.exposure.equalize_adapthist(img) for img in imgs]
    matched_hist = [sk.exposure.match_histograms(img, adpt_norm[match_img]) for img in adpt_norm]
    filtered = []
    for img in matched_hist:
        yen = sk.filters.threshold_yen(img)
        mask = img < yen
        dilate = sk.morphology.dilation(mask)
        filtered.append(sk.morphology.remove_small_objects(sk.morphology.remove_small_holes(dilate,area_threshold=max_hole),min_size=min_object))
    return filtered

#Get dense roi masks within the nuclei and 25-pixel band
def dense_rois_in_nuclei_and_band(dense_roi_masks, band_25_pixels, relabel_nuc_masks):
    dense_roi_in_nuclei = [dense_roi_masks[i] * relabel_nuc_masks[i] for i in range(len(relabel_nuc_masks))]
    dense_roi_in_band = [dense_roi_in_nuclei[i] * band_25_pixels[i] for i in range(len(band_25_pixels))]
    return dense_roi_in_nuclei, dense_roi_in_band

#merge masks and image into array for measurements
def merge_masks_image(imgs,original_nuc_masks,relabel_nuclei_masks,band_25_pixels,dense_roi_in_nuclei,dense_roi_in_band):
    merged_stacks = []
    for i in range(len(imgs)):
        stack = np.stack([imgs[i],original_nuc_masks[i],relabel_nuclei_masks[i],band_25_pixels[i],dense_roi_in_nuclei[i],dense_roi_in_band[i]],axis=-1)
        merged_stacks.append(stack)
    return merged_stacks

#Get heterochromatin and band areas as lists
def heterochromatin_and_band_areas(dense_roi_in_band, band_25_pixels, scale):
    all_heterochromatin_areas = []
    all_band_areas = []
    all_norm_areas = []
    for i in range(len(band_25_pixels)):
        i_heterochromatin_areas = []
        i_band_areas = []
        i_norm_areas = []
        band_props = sk.measure.regionprops(band_25_pixels[i],spacing=scale)
        heterochromatin_props = sk.measure.regionprops(dense_roi_in_band[i],spacing=scale)
        for j in range(len(band_props)):
            band_area = band_props[j].area
            heterochromatin_area = heterochromatin_props[j].area
            norm_area = heterochromatin_area/band_area
            i_heterochromatin_areas.append(heterochromatin_area)
            i_band_areas.append(band_area)
            i_norm_areas.append(norm_area)
        all_heterochromatin_areas.append(i_heterochromatin_areas)
        all_band_areas.append(i_band_areas)
        all_norm_areas.append(i_norm_areas)
    return all_heterochromatin_areas, all_band_areas, all_norm_areas

#Get measurements for all compartments and merge dataframes together (run in loop for all images)
def get_measurements(merged_stacks,props,scale,all_heterochromatin_areas, all_band_areas, all_norm_areas):
    heterochromatin_areas_df = pd.Series(all_heterochromatin_areas,name='heterochromatin_area_in_band_um^2')
    band_areas_df = pd.Series(all_band_areas, name='25-pixel_band_area_um^2')
    norm_area_df = pd.Series(all_norm_areas, name='norm_area_heterochromatin_in_band')
    nuc_measurements_table = sk.measure.regionprops_table(merged_stacks[:,:,2],merged_stacks,props,spacing=scale)
    nuc_regionprops = sk.measure.regionprops(merged_stacks[:,:,2],spacing=scale)
    circularity = []
    for i in range(len(nuc_regionprops)):
        nuc_area = nuc_regionprops[i].area
        nuc_perimeter = nuc_regionprops[i].perimeter
        circularity.append(4*np.pi*nuc_area/nuc_perimeter**2)
    circularity = pd.Series(circularity,name='nuc_circularity')
    nuc_df = pd.DataFrame.from_dict(nuc_measurements_table)
    nuc_df.rename(columns={'label':'object_ID','area':'Nuc_area_um^2','intensity_max-0':'Image_intensity_max', 'intensity_max-1':'Cell_ID',
                           'intensity_max-2':'Nuc_region_ID','intensity_max-3':'Band_ID','intensity_max-4':'Dense_nuc_regions_ID',
                           'intensity_max-5':'Heterochromatin_band_ID','equivalent_diameter_area':'equivalent_diameter_area_um^2',
                           'perimeter':'perimeter_um'},inplace=True)
    df = pd.concat([nuc_df,circularity,heterochromatin_areas_df,band_areas_df,norm_area_df], axis=1)
    return df

#save images as tiffs and dataframes as csv (run in loop for all images along with get measurements function)
def save_df_and_images(df,merged_stacks,save_loc, img_files):
    image_path = os.path.join(save_loc,'merged_imgs')
    df_path = os.path.join(save_loc,'measurements')
    img_name = os.path.basename(img_files)
    imsave(os.path.join(image_path,'merged_stack_'+img_name[:-4]+'.tif'),merged_stacks,check_contrast=False)
    df.to_csv(os.path.join(df_path,'measurements_'+img_name[:-4]+'.csv'))


Load in data and filter out masks touching border and small labeled areas created during ground truth annotation

In [49]:
img_files = sorted(glob("E:/Fondufe-Mittendorf_Lab/TEM_Images/Images/*.tif"))
mask_files = sorted(glob("E:/Fondufe-Mittendorf_Lab/TEM_Images/Masks/*.tif"))
imgs = list(map(imread,img_files))
imgs = [np.asarray(img,dtype=np.uint16) for img in imgs] #make sure all imgs are np compatible arrays
nuclei_masks = list(map(imread,mask_files))
nuclei_masks = [np.asarray(mask,dtype=np.uint16) for mask in nuclei_masks] #make sure all label images are compatible arrays
filtered_nuclei_masks = [sk.segmentation.clear_border(mask) for mask in nuclei_masks]
relabel_nuclei_masks = [sk.measure.label(mask) for mask in filtered_nuclei_masks]
remove_small_objects = [sk.morphology.remove_small_objects(mask, 50) for mask in relabel_nuclei_masks]

E:\Temp\ipykernel_29820\2255796120.py:9: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  remove_small_objects = [sk.morphology.remove_small_objects(mask, 50) for mask in relabel_nuclei_masks]


In [ ]:
footprint = sk.morphology.disk(25)
bands_25 = band_25_pixels(masks=remove_small_objects,footprint=footprint)
dense_roi_masks = norm_and_threshold(imgs)
dense_roi_in_nuclei, dense_roi_in_band = dense_rois_in_nuclei_and_band(dense_roi_masks=dense_roi_masks,band_25_pixels=bands_25,relabel_nuc_masks=remove_small_objects)
merged_stacks = merge_masks_image(imgs,nuclei_masks,remove_small_objects,bands_25,dense_roi_in_nuclei,dense_roi_in_band)
all_heterochromatin_areas, all_band_areas, all_norm_areas = heterochromatin_and_band_areas(dense_roi_in_band,bands_25,scale=[0.0064,0.0064])

KeyboardInterrupt: 

In [ ]:
#collecting measurements for all images
props = ['label', 'area', 'eccentricity', 'equivalent_diameter_area','extent','perimeter','solidity','intensity_max']
scale = [0.0064,0.0064] #in um
save_loc = 'Results/'
for i in range(len(imgs)):
    swapped_axis = np.moveaxis(merged_stacks[i],-1,0)
    df = get_measurements(merged_stacks[i],props=props,scale=scale,all_heterochromatin_areas=all_heterochromatin_areas[i], all_band_areas=all_band_areas[i], all_norm_areas=all_norm_areas[i])
    save_df_and_images(df,swapped_axis,save_loc=save_loc,img_files=img_files[i])